In [2]:
import pandas as pd
import numpy as np

In [3]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\franc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
df = pd.read_csv('thread_df/thread_df_88_42.csv', index_col=0)
comments = df.dropna(subset=['comment'])

In [64]:
sia = SentimentIntensityAnalyzer()
def get_sentiment_score(text):
    return sia.polarity_scores(text)['compound']

In [65]:
comments['sentiment_score'] = comments['comment'].apply(get_sentiment_score)

C:\Users\franc\AppData\Local\Temp\ipykernel_18028\3215188887.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['sentiment_score'] = comments['comment'].apply(get_sentiment_score)


In [66]:
import re
def mark_relevant(dfi):
    df = dfi.copy()
    df['sold_msg'] = df.comment.str.match(pat = '^Sold to .*')
    sold_pos = df.loc[df['sold_msg'] == True].iloc[0,0]
    df['relevant'] = df['position'] >= sold_pos
    df.drop(columns = ['sold_msg'], inplace = True)
    return df

In [67]:
test = mark_relevant(comments)
relevant_comments = test[test['relevant'] == True].iloc[1:]
final_sentiment = relevant_comments.sentiment_score.sum()/len(relevant_comments)